In [ ]:
#Initial load
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from sklearn import metrics
import sys
import GPyOpt
from torchvision import datasets, transforms, utils
from sklearn.model_selection import ParameterSampler, RandomizedSearchCV, cross_val_score
from scipy.stats import uniform
import random

In [ ]:
## define the domain of the considered parameters
#learning_rate = tuple(np.arange(1,11,1, dtype= np.int32)*1e-5)
#learning_rate = tuple([1e-4,2e-4,3e-4,5e-4,6e-4,7e-4,8e-4,9e-4,1e-5])
learning_rate = tuple(np.arange(1,31,0.5)*1e-5)
# Batchsize (discrete, 2-128)
#batch_size = tuple([16,32,64,128,256,16,32,64,128,256])
batch_size = tuple(np.arange(64,128,1))
# num_epochs = (discrete, 1-inf.)
#num_epochs = tuple(np.arange(1,3,1, dtype= np.int32))
#num_epochs = tuple([1,2,3,4,5,6,7,8,9])


# define the dictionary for GPyOpt
domain = [{'name': 'batch_size', 'type': 'discrete', 'domain': batch_size},
          {'name': 'learning_rate', 'type': 'discrete', 'domain':learning_rate}]


## we have to define the function we want to maximize --> validation accuracy, 
## note it should take a 2D ndarray but it is ok that it assumes only one point
## in this setting
def objective_function(x): 
    #print(x)
    # we have to handle the categorical variables that is convert 0/1 to labels
    # log2/sqrt and gini/entropy
    param = x[0]
    print(int(param[0]),param[1])
    # we have to handle the categorical variables
    #if param[2] == 0:
    #    max_f = 'log2'
    #elif param[2] == 1:
       # max_f = 'sqrt'
    #else:
      #  max_f = None

    #if param[3] == 0:
      #  crit = 'gini'
    #else:
       #crit = 'entropy'

    #create the model
    model = Model(n_classes)
    device = torch.device('cuda')  # use cuda or cpu
    model.to(device)
    model2 = CNN(model,device,batch_size = int(param[0]), learning_rate = param[1])
    
    # fit the model 
    
    test_accuracy = model2.training()

    #model.fit(Xtrain, ytrain)
    print(test_accuracy)
    return - test_accuracy


opt = GPyOpt.methods.BayesianOptimization(f = objective_function,   # function to optimize
                                              domain = domain,         # box-constrains of the problem
                                              acquisition_type = 'EI' ,      # Select acquisition function MPI, EI, LCB
                                             )
opt.acquisition.exploration_weight=0.05

opt.run_optimization(max_iter = 15) 

x_best = opt.X[np.argmin(opt.Y)]
print("The best parameters obtained: batch_size=" + str(x_best[0]) + ", learning_rate=" + str(x_best[1]))

In [ ]:
class Bayersian_optimization():
                                            #Domain: [{'name': 'batch_size', 'type': 'discrete', 'domain': batch_size},{}]
                                            #param_list = [batch_size = tuple(np.arange(64,128,1)), learning_rate = tuple(np.arange(1,31,0.5)*1e-5)]


    def __init__(self, domain=None,param_list=[],model=None,device_type='cuda'):
        self.domain = domain
        
        self.param_list = param_list
        self.model = model
        self.device_type = device_type
        super().__init__()                           
    
    def objective_function(self,x):

        #print(x)
        # we have to handle the categorical variables that is convert 0/1 to labels
        # log2/sqrt and gini/entropy
        param = x[0]
        print(int(param[0]),param[1])
        # we have to handle the categorical variables
        #if param[2] == 0:
        #    max_f = 'log2'
        #elif param[2] == 1:
        # max_f = 'sqrt'
        #else:
        #  max_f = None

        #if param[3] == 0:
        #  crit = 'gini'
        #else:
        #crit = 'entropy'

        #create the model
        model = self.model(n_classes)
        device = torch.device(self.device_type)  # use cuda or cpu
        model.to(device)
        model2 = CNN(model,device,batch_size = int(param[0]), learning_rate = param[1])
        
        # fit the model 
        
        test_accuracy = model2.training()

        #model.fit(Xtrain, ytrain)
        print(test_accuracy)
        return - test_accuracy

    def bo(self,max_iter=15,exloration_weight=0.05,acquisition='EI'):
        opt = GPyOpt.methods.BayesianOptimization(f = objective_function,   # function to optimize
                                                    domain = self.domain,         # box-constrains of the problem
                                                    acquisition_type = acquisition,      # Select acquisition function MPI, EI, LCB
                                                    )
        opt.acquisition.exploration_weight=exloration_weight

        opt.run_optimization(max_iter = max_iter) 

        x_best = opt.X[np.argmin(opt.Y)]
        print("The best parameters obtained: batch_size=" + str(x_best[0]) + ", learning_rate=" + str(x_best[1]))
        return x_best, architecture, hyperparameters
